## Coleta de dados para a calculadora de imóveis

In [1]:
import requests

In [2]:
url_ml = 'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/'

In [3]:
resposta = requests.get(url_ml).content

In [4]:
from bs4 import BeautifulSoup

In [5]:
sopa = BeautifulSoup(resposta)

In [6]:
# buscando apenas 1 elemento na sopa
sopa.find('span', {'class':'price-tag-fraction'}).text

'700'

In [7]:
# queremos pegar só o preço de cada elemento do result set que volta da busca da sopa
# só que não conseguimos extrair direto do result set, precisamos fazer um loop
# sopa.find_all('span', {'class': 'price-tag-fraction'}).text

In [8]:
result_set_precos = sopa.find_all('span', {'class': 'price-tag-fraction'})
lista_precos = []

for elemento in result_set_precos:
  lista_precos.append(elemento.text)

In [9]:
lista_precos[:5]

['700', '850', '650', '650', '800']

 ## Tarefa:
 - Fazer o mesmo para áreas e quartos.
 - Se quiser, também fazer para as zonas e mais de uma página do Mercado Livre - as primeiras 5 ou 6 páginas.
 - Se quiserem coletar dados de outras cidades, zonas, de casas, fica a critério de vocês.

In [10]:
lista_area_quartos = []

In [11]:
result_set_area_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')

for elemento in result_set_area_quartos:
  lista_area_quartos.append(elemento.text)

In [12]:
lista_area_quartos[:5]

['58 m² construídos2 quartos',
 '40 m² construídos2 quartos',
 '68 m² construídos2 quartos',
 '30 m² construídos1 quarto',
 '94 m² construídos2 quartos']

In [13]:
result_set_enderecos = sopa.find_all('span', class_='ui-search-item__group__element ui-search-item__location')

In [14]:
result_set_enderecos[:5]

[<span class="ui-search-item__group__element ui-search-item__location">Avenida Paula Ferreira, Vila Pirituba, São Paulo Zona Oeste, São Paulo</span>,
 <span class="ui-search-item__group__element ui-search-item__location">Rua Jaboticabal, Vila Bertioga, São Paulo Zona Leste, São Paulo</span>,
 <span class="ui-search-item__group__element ui-search-item__location">Rua João Infante, Nova Petrópolis, São Bernardo Do Campo, São Paulo</span>,
 <span class="ui-search-item__group__element ui-search-item__location">Praça Marechal Deodoro, Santa Cecília, São Paulo Centro, São Paulo</span>,
 <span class="ui-search-item__group__element ui-search-item__location">Avenida Marechal Deodoro, Gonzaga, Santos, São Paulo</span>]

In [15]:
result_set_enderecos[0].text

'Avenida Paula Ferreira, Vila Pirituba, São Paulo Zona Oeste, São Paulo'

# Coleta de dados de diferentes zonas e páginas

In [16]:
# para cada link fazer o request
# para o request pegar a sopa
# para a sopa extrair as listas (area_quartos, preco, zona, endereco (opcional))
# colocar as listas em um dataFrame (tabelinha)

In [17]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from time import sleep

In [18]:
zonas = ['sul', 'oeste']#, 'leste', 'norte']

In [19]:
paginas = ['', '_Desde_49']#, '_Desde_97', '_Desde_145', '_Desde_193']

In [20]:
def coletando_dados(url, zona):

  resposta = requests.get(url).content
  sopa = BeautifulSoup(resposta)

  result_set_precos = sopa.find_all('span', {'class': 'price-tag-fraction'})
  result_set_area_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')
  result_set_enderecos = sopa.find_all('span', class_='ui-search-item__group__element ui-search-item__location')

  lista_precos = []
  lista_area_quartos = []
  lista_enderecos = []
  lista_zonas = []

  for preco, area_quarto, endereco in zip(result_set_precos, result_set_area_quartos, result_set_enderecos):
    lista_precos.append(preco.text)
    lista_area_quartos.append(area_quarto.text)
    lista_enderecos.append(endereco.text)
    lista_zonas.append(zona)
    
  dados = {'zonas': lista_zonas,
           'areas_quartos': lista_area_quartos,
           'enderecos': lista_enderecos,
           'precos': lista_precos}

  df = pd.DataFrame(dados)
  return df

In [21]:
# # exemplo de try except
# try:
#   for elemento ...
# except:
#   print('deu ruim')

In [22]:
lista_dados = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print('Coletando dados de', url)
    df = coletando_dados(url, zona)
    lista_dados.append(df)
    sleep(2) # lembrar de usar sleep para não sobrecarregar o site de acessos
print('Dados coletados!')

Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-oeste/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-oeste/_Desde_49
Dados coletados!


In [23]:
df_dados = pd.concat(lista_dados)
df_dados

,zonas,areas_quartos,enderecos,precos
0,sul,60 m² construídos2 quartos,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820
1,sul,35 m² construídos1 quarto,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600
2,sul,65 m² construídos3 quartos,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1.492
3,sul,40 m² construídos1 quarto,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1.200
4,sul,40 m² construídos1 quarto,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1.300
...,...,...,...,...
43,oeste,48 m² construídos1 quarto,"Rua Nazira Carone - Jardim Ampliacao, São Paul...",2.000
44,oeste,93 m² construídos2 quartos,"Rua Doutor Luiz Migliano, Jardim Caboré, São P...",3.300
45,oeste,42 m² construídos1 quarto,"Rua Arizona, Cidade Moncoes, São Paulo Zona Oe...",4.000
46,oeste,110 m² construídos3 quartos,"Rua Scipião, Vila Romana, São Paulo Zona Oeste...",5.100


In [24]:
df_dados.to_csv('dados_mercado_livre.csv', index=False) # index=False para tirar o unnamed - coluna adicional de indices

# Manipulando o dataframe

In [25]:
import re

In [26]:
df_ml = pd.read_csv('dados_mercado_livre.csv', thousands='.')

In [27]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos
0,sul,60 m² construídos2 quartos,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820
1,sul,35 m² construídos1 quarto,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600
2,sul,65 m² construídos3 quartos,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1492
3,sul,40 m² construídos1 quarto,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1200
4,sul,40 m² construídos1 quarto,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1300


In [28]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   zonas          192 non-null    object
 1   areas_quartos  192 non-null    object
 2   enderecos      192 non-null    object
 3   precos         192 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 6.1+ KB


In [29]:
linha_0 = df_ml['areas_quartos'][0]
linha_0

'60 m² construídos2 quartos'

In [30]:
# quando não temos a área
linha_0_va = '2 quartos'

In [31]:
def extrair_area(entrada):
  if 'm²' in entrada:
    # uma solução é com o find que devolve o indíce e aí seleciona um pedaço
    # outra solução é dividir o texto onde tem m²
    area = entrada.split(' m²')[0]
  else:
    area = np.nan
  return area

In [32]:
extrair_area(linha_0_va)

nan

In [33]:
extrair_area(linha_0)

'60'

In [34]:
type(np.nan) # 'curiosamente' o nan é um float

float

In [35]:
df_ml['areas'] =  df_ml['areas_quartos'].apply(extrair_area).astype(float) # to_numeric

In [36]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas
0,sul,60 m² construídos2 quartos,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820,60.0
1,sul,35 m² construídos1 quarto,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600,35.0
2,sul,65 m² construídos3 quartos,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1492,65.0
3,sul,40 m² construídos1 quarto,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1200,40.0
4,sul,40 m² construídos1 quarto,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1300,40.0


In [37]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   zonas          192 non-null    object 
 1   areas_quartos  192 non-null    object 
 2   enderecos      192 non-null    object 
 3   precos         192 non-null    int64  
 4   areas          192 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.6+ KB


In [38]:
df_ml['precos'] = df_ml['precos'].astype(float)

In [39]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas
0,sul,60 m² construídos2 quartos,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820.0,60.0
1,sul,35 m² construídos1 quarto,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600.0,35.0
2,sul,65 m² construídos3 quartos,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1492.0,65.0
3,sul,40 m² construídos1 quarto,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1200.0,40.0
4,sul,40 m² construídos1 quarto,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1300.0,40.0


In [40]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   zonas          192 non-null    object 
 1   areas_quartos  192 non-null    object 
 2   enderecos      192 non-null    object 
 3   precos         192 non-null    float64
 4   areas          192 non-null    float64
dtypes: float64(2), object(3)
memory usage: 7.6+ KB


In [41]:
linha_0

'60 m² construídos2 quartos'

In [42]:
linha_0_vaq = '60 m² construídos'

In [43]:
linha_0_vaq

'60 m² construídos'

In [44]:
texto = linha_0
padrao = '(\d+) quarto' # regex que significa um grupo de um ou mais dígitos
padrao1 = '(\d+)'

In [45]:
re.findall(padrao1, texto) # exemplo - aqui já podemos checar e dividir direto em areas e quartos

['60', '2']

In [46]:
re.findall(padrao, texto)[0]

'2'

In [47]:
linha_0

'60 m² construídos2 quartos'

In [48]:
linha_0_vaq

'60 m² construídos'

In [49]:
if 'quarto' in linha_0_vaq:
  n_quartos = re.findall(padrao, linha_0_vaq)[0]
  print(n_quartos)
else:
  print('não temos quartos')

não temos quartos


In [50]:
def extrair_quarto(entrada):

  padrao = '(\d+) quarto'
  
  if 'quarto' in entrada:
    n_quartos = re.findall(padrao, entrada)[0]
  else:
    n_quartos = np.nan
  
  return n_quartos

In [51]:
df_ml['quartos'] = df_ml['areas_quartos'].apply(extrair_quarto).astype(int)

In [52]:
df_ml['quartos']

0      2
1      1
2      3
3      1
4      1
      ..
187    1
188    2
189    1
190    3
191    2
Name: quartos, Length: 192, dtype: int64

In [53]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas,quartos
0,sul,60 m² construídos2 quartos,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820.0,60.0,2
1,sul,35 m² construídos1 quarto,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600.0,35.0,1
2,sul,65 m² construídos3 quartos,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1492.0,65.0,3
3,sul,40 m² construídos1 quarto,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1200.0,40.0,1
4,sul,40 m² construídos1 quarto,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1300.0,40.0,1


In [54]:
df_ml = df_ml.drop('areas_quartos', axis=1) #, inplace=True)
# podemos rodar com o inplace que já executa direto no dado original, aconselho cautela

In [55]:
df_ml.head()

,zonas,enderecos,precos,areas,quartos
0,sul,"Avenida Cupecê, Jardim Prudência, São Paulo Zo...",820.0,60.0,2
1,sul,"Rua Moinho Velho, Moinho Velho, São Paulo Zona...",600.0,35.0,1
2,sul,"Rua Grumixamas, Vila Parque Jabaquara, São Pau...",1492.0,65.0,3
3,sul,"Rua Dos Patriotas, Ipiranga, São Paulo Zona Su...",1200.0,40.0,1
4,sul,"Rua Inglês De Sousa, Jardim Da Glória, São Pau...",1300.0,40.0,1


## Extra: exemplo sobre zip

In [56]:
lista_numeros = [1, 2, 3, 4]
lista_alimentos = ['banana', 'maca', 'feijao', 'arroz']

In [57]:
lista_numeros

[1, 2, 3, 4]

In [58]:
lista_alimentos

['banana', 'maca', 'feijao', 'arroz']

In [59]:
for numero in lista_numeros:
  print(numero)

1
2
3
4


In [60]:
for alimento in lista_alimentos:
  #print(alimento + str(2)) -- essa é uma linha de comentário e ela será ignorada pelo Python
  print(alimento)

banana
maca
feijao
arroz


In [61]:
for numero, alimento in zip(lista_numeros, lista_alimentos):
  print(numero, alimento)

1 banana
2 maca
3 feijao
4 arroz
